In [65]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import datetime
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")

In [66]:
poverty_data_file_path = 'data/preprocessed/poverty_data.geojson'
hex_grid_path = 'data/preprocessed/hex_grid.geojson'
hex_grid_tract_intersections_path = 'data/preprocessed/hex_grid_census_tract_intersections.csv'

In [67]:
poverty_levels_gdf = gpd.read_file(poverty_data_file_path)

In [68]:
print(poverty_levels_gdf.columns)

Index(['Tract Name', 'Tract Population', 'Tract Population Below Poverty',
       'geometry'],
      dtype='object')


In [69]:
def parse_hex_grid():
    gdf = gpd.read_file(hex_grid_path)
    return gdf

In [70]:
hex_grid_gdf = parse_hex_grid()

In [71]:
def parse_grid_tract_data():
    df = pd.read_csv(hex_grid_tract_intersections_path)
    return df

In [72]:
hex_grid_tract_intersections_df = parse_grid_tract_data()

In [73]:
print(hex_grid_tract_intersections_df.columns)

Index(['Grid Id', 'Tract Name', 'Area Fraction', 'Tract Area Fraction'], dtype='object')


In [74]:
print(hex_grid_gdf.columns)

Index(['Grid Id', 'Grid Area', 'Streetlight Count', 'HouseHold Income',
       'Building Count', 'White_Total', 'Black_Total', 'Color_Total',
       '18_Under_Total', '65_Over_Total', 'geometry'],
      dtype='object')


In [75]:
hex_grid_tract_intersections_joined_df = pd.merge(hex_grid_gdf, hex_grid_tract_intersections_df, on='Grid Id', how='inner')

In [76]:
print(hex_grid_tract_intersections_joined_df.columns)

Index(['Grid Id', 'Grid Area', 'Streetlight Count', 'HouseHold Income',
       'Building Count', 'White_Total', 'Black_Total', 'Color_Total',
       '18_Under_Total', '65_Over_Total', 'geometry', 'Tract Name',
       'Area Fraction', 'Tract Area Fraction'],
      dtype='object')


In [62]:
def computeTractPovertyLevel(row):
    return row['Tract Area Fraction'] * poverty_levels_gdf[poverty_levels_gdf['Tract Name'] == row['Tract Name']][['Tract Population', 
             'Tract Population Below Poverty']].reset_index(drop=True)
        
def computePovertyLevel(row):  
    poverty_level = hex_grid_tract_intersections_joined_df[hex_grid_tract_intersections_joined_df['Grid Id'] == row['Grid Id']] \
        .apply(computeTractPovertyLevel, axis=1)
    return poverty_level[poverty_level.index[0]].sum()
    
hex_grid_gdf[['Tract Population', 
             'Tract Population Below Poverty']] = hex_grid_gdf.apply(computeRaceAndAge, axis=1, result_type='expand')

In [63]:
print(hex_grid_gdf[hex_grid_gdf['Grid Id'] ==11])

    Grid Id     Grid Area  Streetlight Count  HouseHold Income  \
11       11  78400.737054                  0     163205.858633   

    Building Count                                           geometry  \
11               6  POLYGON ((-122.41966 47.65473, -122.41801 47.6...   

    White_Total  Black_Total  Color_Total  18_Under_Total  65_Over_Total  
11     53.73766     0.305308    16.835063       15.251601       9.317932  


In [64]:
hex_grid_gdf.to_file('data/preprocessed/hex_grid.geojson', driver="GeoJSON")